# Tabular xAI

## Introduction

**Learning Objectives**

- How to use feature ablation with tabular data
- How to use intergrated gradients to score a features importance

The previous section (and much of this module) focussed on image data, however, the concepts are transferable to tabular data. In this section we will explore how features can be removed to help explain their attribution, as well as how importance can be inferrred by intergrating the gradients of an input given a zero'd baseline.

## Setup

### Data

The dataset used in this demonstration is the Wisconsin **Breast Cancer** Diagnostic Database, publicly available from the [University of California Irvine (UCI) Machine Learning Repository](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic). It consists of characteristics of cell nuclei taken from breast masses which were sampled using fine-needle aspiration (FNA), a common diagnostic procedure in oncology. 

The features are observable characteristics of the cells in the images - for example, radius, concavity, and texture. An example of one of the digitised images from an FNA sample is given below.

![FNA picture](./images/fna_pic.png)

In [ ]:
# Import the modules required:
import torch
import torch.nn as nn

import matplotlib
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy import stats
import pandas as pd
import numpy as np

# fix the numpy and torch seeds for reproducibility:
SEED = 999
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [ ]:
data = pd.read_csv("./data/breast_cancer.csv")

# Change the diagnosis column to binary values
label_dict = {'M': 1, 'B': 0}
data['Diagnosis'] = data['Diagnosis'].map(label_dict)
class_labels = ['Benign', 'Malignant']

print(data.head(5))

# setup oour test train split
X = data.drop("Diagnosis", axis=1)
feature_names = X.columns.tolist() # get feature names
X = X.to_numpy()
y = data["Diagnosis"].to_numpy() 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# rescale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

n_features = X_train.shape[1]
n_classes = len(label_dict)

print(" ")
print(f"Number of features: {n_features}")
print(f"Number of classes: {n_classes}")

The dataset is actually quite small, with only 569 samples. This would usually not be a good candidate for deep learning, but we will use it to demonstrate the process as it is quick to train and has clear, defined features.

## Neural Network Model

We will only use a simple neural netwrod with **one** hidden layer and **sigmoid** activation functions per each layer.

In [ ]:
class simpleNN(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.linear1 = nn.Linear(in_features, 20)
        self.sigmoid1 = nn.Sigmoid()
        self.linear2 = nn.Linear(20, 12)
        self.sigmoid2 = nn.Sigmoid()
        self.linear3 = nn.Linear(12, 2) # binary classification
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        lin1_out = self.linear1(x)
        sigmoid_out1 = self.sigmoid1(lin1_out)
        sigmoid_out2 = self.sigmoid2(self.linear2(sigmoid_out1))
        return self.softmax(self.linear3(sigmoid_out2))

### Training

As the dataset is small and the neurl netwrok simple we can near intantly train the model over 50 epochs, as it converges very quickly.

In [ ]:
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device(f"cuda:0")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f'Device name: {torch.cuda.get_device_name(0)}')
else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")

model = simpleNN(n_features)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=0.1) # adam optimiser

num_epochs = 50

input_tensor = torch.from_numpy(X_train).type(torch.FloatTensor)
label_tensor = torch.from_numpy(y_train)

for epoch in range(num_epochs):    

    output = model(input_tensor)
    loss = criterion(output, label_tensor)
    
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if epoch % 10 == 0:
        print (f'Epoch {epoch+1}/{num_epochs} => Loss: {loss.item():.2f}')

In [ ]:
out_probs = model(input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Train Accuracy:", sum(out_classes == y_train) / len(y_train))

test_input_tensor = torch.from_numpy(X_test).type(torch.FloatTensor)
out_probs = model(test_input_tensor).detach().numpy()
out_classes = np.argmax(out_probs, axis=1)
print("Test Accuracy:", sum(out_classes == y_test) / len(y_test))

Our model is already extremely accurate on both the training and validation datasets. It is therefore a good candidate for understanding how it predicts each of the classes.

## Feature Ablation

The concept of ablating parts of the information in the test dataset can be carried over to tabular data as well. This time, we can use the **FeatureAblation** class to remove features rather than pixels.

TIP BOX

As a default, the ablation class will set the ablated feature to 0. However, it can be modified to be more specific. For example, using the mean of the feature from the training data is a common approach.

```
baseline_mean = torch.tensor(X_train.mean(axis=0)).unsqueeze(0)

attr = ablation.attribute(
    sample,
    baseline=baseline_mean,
    target=predicted_class,
)
```

In [ ]:
from captum.attr import FeatureAblation

In [ ]:
# Get ablation class and set model to evaluation mode
ablation = FeatureAblation(model)
model.eval() # set model to evaluation mode

# Get a single sample and its predicted class
sample = test_input_tensor[2:3]
predicted_class = model(sample).argmax().item()

# Calculate importances
attr = ablation.attribute(
    sample,
    target=predicted_class,
)

plt.figure(figsize=(10, 6))
plt.barh(feature_names, attr[0])
plt.xlabel('Attribution Score')
plt.tight_layout()
plt.show()

Only **texture3** and **radius2** have a positive attribution score for this sample, the remaining are **zero** (which means they have no impact on the prediction). However, looking at the scale these values are very small, indicating that they may also have a minimal impact on the prediction. Due to the variation of just one sample, it is often best to get a more stable attribution by using multiple samples.

Addtionally, we can seperate out the attributions for the positive class (malignant) and the negative class (benign) to see which features are most important for each class.

In [ ]:
# Collect attributions
all_attributions = {0: [], 1: []}

for class_idx in range(n_classes):
    
    # Get samples from this class
    class_mask = y_test == class_idx
    class_samples = X_test[class_mask]
    
    for sample in class_samples:
        sample_tensor = torch.tensor(sample, dtype=torch.float32).unsqueeze(0)
        attr = ablation.attribute(sample_tensor, target=class_idx)
        all_attributions[class_idx].append(attr.squeeze().cpu().numpy())

# Average attributions per class
avg_attributions_malignant = np.mean(all_attributions[1], axis=0)
avg_attributions_benign = np.mean(all_attributions[0], axis=0)
# Find features that are important for one class but not the other
diff_importance = avg_attributions_malignant - avg_attributions_benign

plt.figure(figsize=(10, 6)) 
colors = ['red' if x < 0 else 'green' for x in diff_importance]
plt.barh(feature_names, diff_importance, color=colors)
plt.xlabel('Importance Difference (Malignant - Benign)')
plt.axvline(x=0, color='black', linestyle='--') # plot dashed line at 0
plt.tight_layout()
plt.show()

Finding the average importance of each feature gives a much richer view of what features are most important to the model. According to this representation, most of the features are postively correlated with the target variable, malignant. **Symmetry3** has the highest score and would be a good candidate for further breaking down what range of values are most important within that feature.

## Intergrated Gradients

An alternative to feature ablation is to calculate how much each feature contributes through their integrated gradients. This method measures how much each feature contributes to the final prediction by computing gradients along a path from a baseline (e.g., zeros or average values) to the actual input.

Rather than simply removing features, Integrated Gradients creates many intermediate points between the baseline and your input, calculates how sensitive the model is to each feature at each point, and then averages these sensitivities. 

This gradient-based approach is often more efficient than ablation for high-dimensional data.

In [ ]:
from captum.attr import IntegratedGradients

In [ ]:
ig = IntegratedGradients(model)

test_input_tensor.requires_grad_()

attr, delta = ig.attribute(test_input_tensor, 
                            target=1,                      # set the target class to 1 (malignant)
                            return_convergence_delta=True
                            )

attr = attr.detach().numpy()

attr_means = np.mean(attr, axis=0)

By setting the target to 1, we are asking for features that are most important (positve attribution) to predicting the malignant class or (if negative) have the opposite effect.

In [ ]:
def visualise_importances(feature_names, importances, title="", xaxis_title="Feature Attribution"):
    """
        Plotting function for feature importances
    """    

    plt.figure(figsize=(10, 6)) 

    colors = ['red' if x < 0 else 'green' for x in importances]
    plt.barh(feature_names, importances, color=colors)
    plt.xlabel(xaxis_title)
    plt.axvline(x=0, color='black', linestyle='--') # plot dashed line at 0
    plt.tight_layout()
    plt.show()

visualise_importances(feature_names, attr_means)

positive_feature = feature_names[np.argmax(attr_means)]
all_best = attr[:, np.argmax(attr_means)]
print(f"Feature most postively attributed to malignancy: {positive_feature}")

negative_feature = feature_names[np.argmin(attr_means)]
all_worst = attr[:, np.argmin(attr_means)]
print(f"Feature most negatively attributed to malignancy: {negative_feature}")

### Distribution of scores

Means of scores can often be misleading, especially for feature attributions methods where **zero** indicates no contribtution to the prediciton either positively or negatively. Plotting a histogram of the attributions can give a better sense of the distribution.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].hist(all_best);
ax[0].set_title(f"Distribution of most positive correlated feature: \n {positive_feature}");

ax[1].hist(all_worst);
ax[1].set_title(f"Distribution of most negative correlated feature: \n {negative_feature}");

fig.tight_layout()

plt.show()

Both distributions for the most positive and negative predictions are centered around 0 as most samples have a feature attribution score of `0`. This indicates that in the majority of cases, these features do not contribute much at all. This could be likely for all features.

However, there are still the outliers where the feature is very influencial. To see when this is occuring we can bin the values in te feature and plot the average attribution at differing values.

In [ ]:
# Plot the scores 
best_positive_idx = np.argmax(attr_means)
best_negative_idx = np.argmin(attr_means)

bin_means, bin_edges, _ = stats.binned_statistic(X_test[:,best_positive_idx], attr[:,best_positive_idx], statistic='mean', bins=6) # change values into bins
bin_count, _, _ = stats.binned_statistic(X_test[:,best_positive_idx], attr[:,best_positive_idx], statistic='count', bins=6) # get the cound of each bin 

bin_means_neg, bin_edges_neg, _ = stats.binned_statistic(X_test[:,best_negative_idx], attr[:,best_negative_idx], statistic='mean', bins=6) # change values into bins
bin_count_neg, _, _ = stats.binned_statistic(X_test[:,best_negative_idx], attr[:,best_negative_idx], statistic='count', bins=6) # get the cound of each bin 

bin_width = (bin_edges[1] - bin_edges[0])
bin_centers = bin_edges[1:] - bin_width/2

bin_width_neg = (bin_edges_neg[1] - bin_edges_neg[0])
bin_centers_neg = bin_edges_neg[1:] - bin_width_neg/2

fig, ax = plt.subplots(1, 2, figsize=(12, 5))

ax[0].scatter(bin_centers, bin_means, s=bin_count) # change the size given the coutn
ax[0].set_ylabel("Average Attribution");
ax[0].set_title(f"Binned attribution for positively correlated feature \n {positive_feature}");


ax[1].scatter(bin_centers_neg, bin_means_neg, s=bin_count_neg) # change the size given the coutn
ax[1].set_ylabel("Average Attribution");
ax[1].set_title(f"Binned attribution for negatively correlated feature \n {negative_feature}");

plt.show()

Breaking the scores down per bins of values reveals information about how the data contributes to the prediction. For the positivly correleated feature, **concavity3**, the average attribution rises as concavity3 increases, indicating that higher values of concavity3 contribute to higher predictions. Values over **2** are extemely more perdictive of a malignant diagnosis.

For **compactness2** the average attrbution stays approximately the same until the value rises to `**5**`, when it jumps to a score of `**-5**`.

The further breaking down of each feature allows us to extract meaningful, biological information, that can be utilised by a proffessional to better guide their understanding of the output or their research.

## Other Methods

**Captum** has several other methods for better understanding neural networks. Two that are prominently used are the [**Layer Attribtuions**](https://captum.ai/api/layer.html) class and the [**Neurons Attribtuions**](https://captum.ai/api/neuron.html) class. These facilitate insight into how each layer in the neural network contributed to the final prediction and how each feature inputs into each neuron.

**Layer Attributions** allow you to examine the importance of how each layer of your network are most relevant for a prediction. This is particularly valuable for debugging deep networks and understanding what patterns the model has learned at different levels of abstraction.

**Neuron Attributions** go deeper by showing how individual neurons respond to specific inputs, revealing which input features activate particular neurons and how strongly. This granular view helps identify whether neurons have learned meaningful patterns or spurious correlations, and can guide network architecture improvements.

Combined, these two methods can help break down every component of the model to best understand how it is giving predictions. They enable you to trace the flow of information from raw inputs through hidden layers to final outputs, making it possible to identify exactly where and how the model processes different types of information.

## Summary

Gaining meaningful insights from machine learning models can be arduous, relying on domain knowledge combined with a deep dive into how the scale of values of a feature contribute to a prediction. Even in the cases where you can find a feature that is important to a prediction, you must remember that it is still a correlation, not a causation. There have been many cases of models with accurate predictions, but were fundamentally flawed due to biases in their training data. It was up do researchers to identify these biases and rectify them, not to confirm them as true.